<a href="https://colab.research.google.com/github/Kangjunseo/NLP/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/NLP")

In [4]:
import pandas as pd
df=pd.read_csv('ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [5]:
headline = [] # 리스트 선언
headline.extend(list(df.headline.values)) # 헤드라인의 값들을 리스트로 저장
headline[:5] # 상위 5개만 출력

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [6]:
len(headline) # 현재 데이터의 길이

1324

In [7]:
headline = [n for n in headline if n != "Unknown"] # Unknown 데이터 제거
len(headline) # 제거 후 데이터의 길이

1214

In [8]:
from string import punctuation
def preprocessing(s):
    return ''.join(c for c in s if c not in punctuation).lower() # 구두점 제거와 동시에 소문자화

text = [preprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders’ settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [9]:
from string import punctuation
def repreprocessing(s):
    s=s.encode("utf8").decode("ascii",'ignore') # 코드 추가
    return ''.join(c for c in s if c not in punctuation).lower() # 구두점 제거와 동시에 소문자화

text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [10]:
from keras_preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [12]:
sequences = list()

for line in text: # 1214 개의 데이터에 대해서 데이터를 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 데이터에 대한 정수 인코딩
    
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:5]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52]]

In [13]:
max_len=max(len(l) for l in sequences)
print(max_len)

24


In [14]:
from keras.preprocessing.sequence import pad_sequences
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

Using TensorFlow backend.


[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


padding='pre'를 설정하면 데이터의 길이가 24보다 짧은 경우에는 앞에 0으로

In [0]:
import numpy as np
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [16]:
print(X)

[[   0    0    0 ...    0    0   99]
 [   0    0    0 ...    0   99  269]
 [   0    0    0 ...   99  269  371]
 ...
 [   0    0    0 ...  170    8 3493]
 [   0    0    0 ...    8 3493  115]
 [   0    0    0 ... 3493  115    2]]


In [0]:
from keras.utils import to_categorical
y = to_categorical(y, num_classes=vocab_size)

In [18]:
print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [20]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4973093050002506886
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4065842990651980477
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12392585855594572980
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14800692839
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1557106992668262239
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [21]:
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
# y를 제거하였으므로 이제 X의 길이는 기존 데이터의 길이 - 1
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
 - 9s - loss: 7.6605 - acc: 0.0250
Epoch 2/200
 - 8s - loss: 7.1240 - acc: 0.0274
Epoch 3/200
 - 8s - loss: 6.9902 - acc: 0.0372
Epoch 4/200
 - 8s - loss: 6.8619 - acc: 0.0424
Epoch 5/200
 - 8s - loss: 6.7130 - acc: 0.0447
Epoch 6/200
 - 9s - loss: 6.5425 - acc: 0.0469
Epoch 7/200
 - 8s - loss: 6.3560 - acc: 0.0527
Epoch 8/200
 - 8s - loss: 6.1616 - acc: 0.0578
Epoch 9/200
 - 8s - loss: 5.9657 - acc: 0.0609
Epoch 10/200
 - 8s - loss: 5.7722 - acc: 0.0657
Epoch 11/200
 - 8s - loss: 5.5832 - acc: 0.0697
Epoch 12/200
 - 8s - loss: 5.4090 - acc: 0.0752
Epoch 13/200
 - 9s - loss: 5.2398 - acc: 0.0832
Epoch 14/200
 - 10s - loss: 5.0792 - acc: 0.0925
Epoch 15/200
 - 9s - loss: 4.9270 - acc: 0.0984
Epoch 16/200
 - 9s - loss: 4.7804 - acc: 0.1135
Epoch 17/200
 - 8s - loss: 4.6378 - acc: 0.1260
Epoch 18/200
 - 8s - loss: 4.5025 - acc: 0.1385
Epoch 19/200
 - 8s - loss: 4.3687 - acc: 0.1616
Epoch 20/200
 - 8s - loss: 4.2404 - acc: 0.1794
Epoch 21/200
 - 8s - loss: 4.1172 - acc: 0.1942
